In [2]:
import psycopg2 as psycopg
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ.get('DB_DESTINATION_HOST'), 
    "port": os.environ.get('DB_DESTINATION_PORT'),
    "dbname": os.environ.get('DB_DESTINATION_NAME'),
    "user": os.environ.get('DB_DESTINATION_USER'),
    "password": os.environ.get('DB_DESTINATION_PASSWORD'),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

In [3]:
# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [25]:
# 1. Название колонок вашего датафрейма запишите в текстовый файл
with open("columns.txt", "w", encoding="utf-8") as fio:
    i=1
    for col in df.columns:
        if i == df.columns.shape[0]:
            fio.write(col)
        else: 
            if col != 'id':
                fio.write(col+',')
        i+=1
#df.columns.to_csv('columns1.txt', index=False)
          

In [27]:
# второй вариант
import csv  
#columns1 = [columns]
# 1. Название колонок вашего датафрейма запишите в текстовый файл
with open("columns4.txt", "w", encoding="utf-8") as fio:
    writer = csv.writer(fio) 
    # Записываем все данные за один раз  
    writer.writerows([df.columns])

In [ ]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}
for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}
    stats.update(column_stat)

stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()
stats["unique_customers_number"] = df["customer_id"].nunique()
stats["end_date_nan"] = df["end_date"].isna().sum()

{'Month-to-month': 3875, 'Two year': 1695, 'One year': 1473}
{'Yes': 4171, 'No': 2872}
{'Fiber optic': 3096, 'DSL': 2421}
{'No': 3498, 'Yes': 2019}
{'No': 3088, 'Yes': 2429}
{'No': 3095, 'Yes': 2422}
{'No': 3473, 'Yes': 2044}
{'No': 2810, 'Yes': 2707}
{'No': 2785, 'Yes': 2732}
{'Male': 3555, 'Female': 3488}
{0: 5901, 1: 1142}
{'No': 3641, 'Yes': 3402}
{'No': 4933, 'Yes': 2110}
{'No': 3390, 'Yes': 2971}
{0: 5174, 1: 1869}


{'Month-to-month': 3875, 'Two year': 1695, 'One year': 1473}